In [2]:
import anndata 
import pandas as pd
import scanpy as sc
import pybedtools
import numpy as np
from ALLCools.plot import categorical_scatter,continuous_scatter
import matplotlib.pyplot as plt
from ALLCools.clustering import \
    tsne, \
    significant_pc_test, \
    filter_regions, \
    remove_black_list_region, \
    lsi, \
    binarize_matrix
import joblib
from amb import *
from ALLCools.clustering import *


plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi']=300
plt.rcParams['pdf.fonttype']=42

age_palette = get_age_palette()
age_palette['8wk'] = age_palette['2mo']

ct_palette = joblib.load('/home/qzeng_salk_edu/project/240319_te/AgingMajorType_Palette')
# replace the keys by .apply(lambda x: x.replace(' ','_').replace('/',''))
ct_palette = {x.replace(' ','_').replace('/',''):y for x,y in ct_palette.items()}

region_palette = get_major_region_palette()

In [3]:
#group_name = 'CA1-ProS_Glut'

In [4]:
mc_type = 'CGN'
te_class = 'LINE'
inout='in'
ct = group_name

In [5]:
genomepath = '/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
gene_bed = pybedtools.BedTool('/home/qzeng_salk_edu/project/241115_te_clustering/TE_use.gencode.vM22.annotation.gene.sorted.bed')
te_bed = pybedtools.BedTool(f'/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-{te_class}.bed')

In [6]:
meta = pd.read_csv('/data/metadata/240104_m3C_META.csv', index_col =0)
meta['AgingMajorType'] = meta['AgingMajorType'].apply(lambda x: x.replace(' ','_').replace('/',''))
meta.head()

,InputReadPairs,InputReadPairsBP,TrimmedReadPairs,R1WithAdapters,R1QualTrimBP,R1TrimmedReadsBP,R2WithAdapters,R2QualTrimBP,R2TrimmedReadsBP,UniqueMappedReads,...,DissectionRegion,Region,Replicate,l1,mc_m3c-co_cluster_l1,mc_m3c-co_cluster_l2,CellType,CellSubclass,AgingMajorType,Class
cell,,,,,,,,,,,,,,,,,,,,,
AMB_220628_18mo_3F_4E_1_P1-1-I3-A1,1710943,516615812,1710512,309,766343,216480647,874,649089,230238965,1870858,...,ACB-1+ACB-2,Nucleus_accumbens,rep2,13,23,0-0,STR D1 Gaba,STR D1 Gaba,STR_D1_Gaba,Gaba
AMB_220628_18mo_3F_4E_1_P1-1-I3-A13,1688475,510130057,1688130,316,725418,213815592,832,673498,227339515,1820523,...,ACB-1+ACB-2,Nucleus_accumbens,rep2,11,6,12-6,STR-PAL Chst9 Gaba,STR D1 Sema5a Gaba,STR_D1_Sema5a_Gaba,Gaba
AMB_220628_18mo_3F_4E_1_P1-1-I3-A14,2158376,651979882,2157941,456,998559,273186730,1132,869248,290536436,2374173,...,ACB-1+ACB-2,Nucleus_accumbens,rep2,8,4,2-2,Astro NN,Astro-TE NN,Astro-TE_NN,NN
AMB_220628_18mo_3F_4E_1_P1-1-I3-A2,1582810,478040623,1582446,283,708122,200329678,853,626267,213020525,1746308,...,ACB-1+ACB-2,Nucleus_accumbens,rep2,9,8,7-6,STR D2 Gaba,STR D2 Gaba,STR_D2_Gaba,Gaba
AMB_220628_18mo_3F_4E_1_P1-1-I3-B1,1766714,533610284,1766279,411,810795,223604054,1045,691737,237777442,1968711,...,ACB-1+ACB-2,Nucleus_accumbens,rep2,13,23,0-0,STR D1 Gaba,STR D1 Gaba,STR_D1_Gaba,Gaba


In [14]:
plot_inouts  = ['out20','out2','out','all','in'] #'out10',

In [15]:
ads =  anndata.read_h5ad(f"/home/qzeng_salk_edu/project/241115_te_clustering/adata/celltype/{ct}.{te_class}.downsampled.h5ad")
ads.obs['celltype'] = meta['AgingMajorType']
ads.obs['Age'] = meta['Age']
ads.obs['Region'] = meta['Region']

# change ads.X to df
df = pd.DataFrame(ads.X.toarray(), index=ads.obs_names, columns=ads.var_names)
ads.obs['hypo_te_counts'] = df.sum(axis = 1)
ads

AnnData object with n_obs × n_vars = 4851 × 954358
    obs: 'celltype', 'Age', 'Region', 'hypo_te_counts'

In [16]:
for inout in plot_inouts:
    if inout=='out20':
        feats = te_bed.subtract(gene_bed.flank(b=20000, g=genomepath)).to_dataframe()['name'].tolist()
    elif inout=='out10':
        feats = te_bed.subtract(gene_bed.flank(b=10000, g=genomepath)).to_dataframe()['name'].tolist()
    elif inout=='out2':
        feats = te_bed.subtract(gene_bed.flank(b=2000, g=genomepath)).to_dataframe()['name'].tolist()
    elif inout=='out':
        feats = te_bed.subtract(gene_bed).to_dataframe()['name'].tolist()
    elif inout=='in':
        feats = te_bed.intersect(gene_bed,).to_dataframe()['name'].tolist()
    else:
        feats = te_bed.to_dataframe()['name'].tolist()

    shared_feats = list(set(feats) & set(ads.var_names))
    print(f"{inout}:{len(shared_feats)}slected feats")

    # original adata

    cluster_ads = ads[:,shared_feats].copy()

    sc.pp.filter_genes(cluster_ads, min_cells=3)

    pc_cutoff = 0.1
    lsi(cluster_ads, algorithm='arpack', obsm='X_pca')
    n_components = significant_pc_test(cluster_ads, p_cutoff=pc_cutoff, update=True)

    resolution = 0.5
    sc.pp.neighbors(cluster_ads)
    sc.tl.leiden(cluster_ads, resolution=resolution)
    sc.tl.umap(cluster_ads)

    tsne(cluster_ads,
        obsm='X_pca',
        metric='euclidean',
        exaggeration=-1,  # auto determined
        perplexity=30,
        n_jobs=-1)

    # add the f'{inout}_tsne_1 and f'{inout}_tsne_2 in obs to obsm
    ads.obsm[f'X_{inout}_tsne'] = cluster_ads.obsm['X_tsne']
    ads.obs[f"{inout}_leiden"] = cluster_ads.obs['leiden']
    print(f"{inout} tsne done")
    
ads.write_h5ad(f"clustered.{ct}.{te_class}.h5ad")




out20:546795slected feats
5 components passed P cutoff of 0.1.
Changing adata.obsm['X_pca'] from shape (4851, 100) to (4851, 5)
out20 tsne done
in:353424slected feats
6 components passed P cutoff of 0.1.
Changing adata.obsm['X_pca'] from shape (4851, 100) to (4851, 6)
in tsne done


## plot

In [19]:
def plot_tsne(ads, coord_base, inout, **kwargs):
    fig, axes = plt.subplots(figsize=(8, 8),ncols = 2,nrows = 2,  dpi=300, constrained_layout=True)
    ax = axes[0,0]
    categorical_scatter(data=ads,
                        ax=ax, 
                        coord_base=coord_base, 
                        hue=f'{inout}_leiden', 
                        rasterized= True,
                        text_anno = f'{inout}_leiden',
                        show_legend=False
                        )

    ax.set_title(f"leiden resolution={resolution}")

    ax = axes[0,1]
    categorical_scatter(data=ads, 
                        ax=ax, 
                        coord_base=coord_base, 
                        hue='Region', 
                        #text_anno = 'Region',
                        rasterized= True,
                        show_legend=True, 
                        palette=region_palette
                        )
    ax.set_title(f"Region")                    

    ax = axes[1,0]
    categorical_scatter(data=ads, 
                    ax=ax, 
                    coord_base=coord_base, 
                    hue='Age', 
                    #text_anno = 'Age',
                    rasterized= True,
                    show_legend=True, 
                    palette=age_palette)
    ax.set_title(f"Age")  

    ax = axes[1,1]
    continuous_scatter(data=ads, 
                    ax=ax, 
                    coord_base=coord_base, 
                    hue='hypo_te_counts', 
                    hue_norm=(np.percentile(ads.obs['hypo_te_counts'], 5), np.percentile(ads.obs['hypo_te_counts'], 95)),
                    )
    ax.set_title(f"hypo(frac<0.75) te counts")  
    plt.suptitle(f"{te_class} {ct} {inout}")
    
    return fig
    #fig.savefig(f"figure/{ct}.{te_class}.{inout}.tsne.pdf")

In [20]:
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages(f'{ct}_{te_class}_pdf.pdf') as pdf:
    for inout in plot_inouts:
        fig = plot_tsne(ads, 
            coord_base = f'{inout}_tsne', 
            inout = inout)
        pdf.savefig(fig)  # or you can pass a Figure object to pdf.savefig
        plt.close()

/opt/conda/lib/python3.10/site-packages/ALLCools/plot/text_anno_scatter.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for text, sub_df in data.groupby(anno_col):
/opt/conda/lib/python3.10/site-packages/ALLCools/plot/text_anno_scatter.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for text, sub_df in data.groupby(anno_col):


In [12]:
# select high vraiable feature among Age
#sc.pp.highly_variable_genes(ads, n_top_genes=2000, flavor='seurat', n_bins=20)

# # cluster_enriched_features(ads,
# #                           cluster_col='Age',
# #                           top_n=200,
# #                           alpha=0.05,
# #                           stat_plot=True,
# #                           method='mc')

# cef = ads.var[f'Age_enriched_features'].copy()
# cef.sum()

# cef = ads.var[f'Age_enriched_features'].copy()
# ads = ads[:,cef]